In [ ]:
# Load repo (if needed)
!git clone https://github.com/JoePenna/Dreambooth-Stable-Diffusion
%cd Dreambooth-Stable-Diffusion

In [ ]:
# BUILD ENV
!pip install omegaconf
!pip install einops
!pip install pytorch-lightning==1.6.5
!pip install test-tube
!pip install transformers
!pip install kornia
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install setuptools==59.5.0
!pip install pillow==9.0.1
!pip install torchmetrics==0.6.0
!pip install -e .
!pip install protobuf==3.20.1
!pip install gdown
!pip install pydrive
!pip install -qq diffusers["training"]==0.3.0 transformers ftfy
!pip install -qq "ipywidgets>=7,<8"
!pip install huggingface_hub
!pip install ipywidgets==7.7.1

In [1]:
# Hugging Face Login
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Download the 1.4 sd model
from IPython.display import clear_output

from huggingface_hub import hf_hub_download
downloaded_model_path = hf_hub_download(
 repo_id="CompVis/stable-diffusion-v-1-4-original",
 filename="sd-v1-4.ckpt",
 use_auth_token=True
)

# Move the sd-v1-4.ckpt to the root of this directory as "model.ckpt"
actual_locations_of_model_blob = !readlink -f {downloaded_model_path}
!mv {actual_locations_of_model_blob[-1]} model.ckpt
clear_output()
print("✅ model.ckpt successfully downloaded")


# Download Regularization Images
We’ve created the following image sets
- `man_euler` - provided by Niko Pueringer (Corridor Digital) - euler @ 40 steps, CFG 7.5
- `man_unsplash` - pictures from various photographers
- `person_ddim`
- `woman_ddim` - provided by David Bielejeski - ddim @ 50 steps, CFG 10.0 <br />
- `blonde_woman` - provided by David Bielejeski - ddim @ 50 steps, CFG 10.0 <br />

In [2]:
dataset="person_ddim"
!git clone https://github.com/djbielejeski/Stable-Diffusion-Regularization-Images-{dataset}.git

!mkdir -p regularization_images/{dataset}
!mv -v Stable-Diffusion-Regularization-Images-{dataset}/{dataset}/*.* regularization_images/{dataset}

# Create the training image directory
!rm -rf training_images
!mkdir -p training_images

# Training Images
## Upload your training images
WARNING: Be sure to upload an even amount of images, otherwise the training inexplicably stops at 1500 steps. <br />
- 2-3 full body
- 3-5 upper body
- 5-12 close-up on face  <br /> <br />
The images should be as close as possible to the kind of images you’re trying to make (most of the time, that means no selfies).

In [ ]:
from IPython.display import clear_output

# Tell the user what is going on
training_images_file_paths = !find training_images/*
if len(training_images_file_paths) == 0:
 print("❌ no training images found. Please upload images to training_images")
else:
 print("✅ " + str(len(training_images_file_paths)) + " training images found")


In [ ]:
# Training

# This isn't used for training, just to help you remember what your trained into the model.
project_name = "project_name"

# MAX STEPS
# How many steps do you want to train for?
max_training_steps = 2000

# Match class_word to the category of the regularization images you chose above.
class_word = "person"

# This is the unique token you are incorporating into the stable diffusion model.
token = "firstNameLastName"
reg_data_root = "/workspace/Dreambooth-Stable-Diffusion/regularization_images/" + dataset

!rm -rf training_images/.ipynb_checkpoints
!python "main.py" \
 --base configs/stable-diffusion/v1-finetune_unfrozen.yaml \
 -t \
 --actual_resume "model.ckpt" \
 --reg_data_root "{reg_data_root}" \
 -n "{project_name}" \
 --gpus 0, \
 --data_root "/workspace/Dreambooth-Stable-Diffusion/training_images" \
 --max_training_steps {max_training_steps} \
 --class_word "{class_word}" \
 --token "{token}" \
 --no-test

In [ ]:
# Copy and name the checkpoint file

directory_paths = !ls -d logs/*
last_checkpoint_file = directory_paths[-1] + "/checkpoints/last.ckpt"
training_images = !find training_images/*
date_string = !date +"%Y-%m-%dT%H-%M-%S"
file_name = date_string[-1] + "_" + project_name + "_" + str(len(training_images)) + "_training_images_" +  str(max_training_steps) + "_max_training_steps_" + token + "_token_" + class_word + "_class_word.ckpt"

file_name = file_name.replace(" ", "_")

!mkdir -p trained_models
!mv "{last_checkpoint_file}" "trained_models/{file_name}"

print("Download your trained model file from trained_models/" + file_name + " and use in your favorite Stable Diffusion repo!")

# Optional - Upload to google drive
* run the following commands in a new `terminal` in the `Dreambooth-Stable-Diffusion` directory
* `chmod +x ./gdrive`
* `./gdrive about`
* `paste your token here after navigating to the link`
* `./gdrive upload trained_models/{file_name.ckpt}`

# Big Important Note!

The way to use your token is `<token> <class>` ie `joepenna person` and not just `joepenna`

## Generate Images With Your Trained Model!

In [ ]:
model = f"/workspace/Dreambooth-Stable-Diffusion/trained_models/{file_name}" 
prompt =  f"{token} {class_word} person as a masterpiece portrait painting by John Singer Sargent in the style of Rembrandt"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples 4 \
 --n_iter 1 \
 --scale 7.0 \
 --ddim_steps 50 \
 --ckpt "{model}" \
 --prompt "{prompt}"